# Text Classification Slides - Deep Learning - Kiva

Predict whether a Kiva loan application will default.

- Stephen W. Thomas
- Used for MMAI 891 and MMA/GMMA 865.
- The Deep Learning version.

# Preliminaries: Inspect and Set up environment

In [1]:
!which python

/usr/local/bin/python


In [2]:
!python --version

Python 3.6.9


In [3]:
!echo $PYTHONPATH

/env/python


In [4]:
import keras
print(keras.__version__)

2.4.3


In [5]:
import sklearn
print(sklearn.__version__)

0.22.2.post1


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from keras.models import Model, Sequential
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, Bidirectional
from keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [7]:
df = pd.read_csv('https://raw.githubusercontent.com/stepthom/NLP_course/main/data/sentiment_train.csv', delimiter=',', encoding='latin-1')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400 entries, 0 to 2399
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sentence  2400 non-null   object
 1   Polarity  2400 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 37.6+ KB


In [8]:
df.head()

,Sentence,Polarity
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [9]:
from sklearn.preprocessing import LabelEncoder

X = df.Sentence
y = df.Polarity
le = LabelEncoder()
y = le.fit_transform(y)
y = y.reshape(-1,1)

In [10]:
y.shape
y

(2400, 1)

array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [1]])

In [11]:
X.shape
X

(2400,)

0                                Wow... Loved this place.
1                                      Crust is not good.
2               Not tasty and the texture was just nasty.
3       Stopped by during the late May bank holiday of...
4       The selection on the menu was great and so wer...
                              ...                        
2395    Almost all of the songs in Cover Girl are old-...
2396    The most annoying thing about 'Cover Girl' is ...
2397    Unfortunately, 'Cover Girl' is an example of h...
2398    Non-linear narration thus many flashbacks and ...
2399    The good cinematography also makes her and Mon...
Name: Sentence, Length: 2400, dtype: object

In [12]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train, y_test = train_test_split (X, y,  test_size=0.15)

In [13]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences, maxlen=max_len)

In [14]:
sequences_matrix.shape
sequences_matrix[0,:]

(2040, 150)

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0, 537,   2,  18, 277], dtype=int32)

In [15]:
model = Sequential()

model.add(Embedding(max_words, 32, input_length=max_len))
model.add((LSTM(64, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(1, name='out_layer', activation='sigmoid'))

model.summary()
model.compile(loss='binary_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 150, 32)           32000     
_________________________________________________________________
lstm (LSTM)                  (None, 64)                24832     
_________________________________________________________________
out_layer (Dense)            (None, 1)                 65        
Total params: 56,897
Trainable params: 56,897
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.fit(sequences_matrix, y_train, batch_size=128, epochs=100,
          validation_split=0.2, callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Epoch 1/100
13/13 [==============================] - 4s 328ms/step - loss: 0.3226 - accuracy: 0.8744 - val_loss: 0.4754 - val_accuracy: 0.8088
Epoch 2/100
13/13 [==============================] - 5s 382ms/step - loss: 0.3325 - accuracy: 0.8830 - val_loss: 0.4756 - val_accuracy: 0.8260


In [21]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences, maxlen=max_len)

In [22]:
accr = model.evaluate(test_sequences_matrix, y_test)

12/12 [==============================] - 0s 32ms/step - loss: 0.4497 - accuracy: 0.7917


In [23]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.450
  Accuracy: 0.792
